# Data Modelling

In [4]:
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.regression import GeneralizedLinearRegression
from helpers.helper_functions import translate_to_file_string
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

inputFile = translate_to_file_string("./data/Data_Preparation_Result.csv")

def prettyPrint(dm, collArray) :
    rows = dm.toArray().tolist()
    dfDM = spark.createDataFrame(rows,collArray)
    newDf = dfDM.toPandas()
    from IPython.display import display, HTML
    return HTML(newDf.to_html(index=False))

## Create Spark Session

In [5]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("DataModelling")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- Bundesland: string (nullable = true)
 |-- BundeslandIndex: integer (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- LandkreisIndex: integer (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- AltersgruppeIndex: double (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- GeschlechtIndex: double (nullable = true)
 |-- FallStatus: string (nullable = true)
 |-- FallStatusIndex: double (nullable = true)
 |-- Falldatum: string (nullable = true)

None


## Vorbereitung der Daten

### Filtern der Datensätze
Für das Training dieses Modells ist es sinnvoll nur die Fälle zu betrachten, bei den der Ausgang der Corona-Erkrankung bereits bekannt ist ("GENESEN" oder "GESTORBEN"). Daher werden die Fälle mit noch erkrankten Personen herausgefiltert. Ebenfalls muss der FallStatusIndex neu vergeben werden, damit dieses Feature nur noch die Werte 0 oder 1 enthält.

In [6]:
dfNeu = df.filter(df.FallStatus != "NICHTEINGETRETEN").drop("FallStatusIndex")

### FallStatusIndex

In [7]:
indexer = StringIndexer(inputCol="FallStatus", outputCol="FallStatusIndex")
dfReindexed = indexer.fit(dfNeu).transform(dfNeu)

### Ziehen eines Samples
Da der Datensatz sehr groß ist,kann es evt. notwendig sein, nur mit einem kleineren Umfang zu trainieren. Mit Fraction kann an dieser Stelle der Umfang der Stichprobe angepasst werden.

In [8]:
dfsample = dfReindexed.sample(withReplacement=False, fraction=1.0, seed=12334556)

### Undersampling
Ähnlich dem Fraud-Detection-Beispiel von Tara Boyle (2019) ist die Klasse der an Corona-Verstorbenen im vorliegenden Datensatz unterrepresentiert, weshalb man an dieser Stelle von einer Data Imbalance spricht. Dies sieht man wenn man die Anzahl der Todesfälle mit den Anzahl der Genesenen vergleicht.

In [9]:
# Vergleich der Fallzahlen
dfsample.groupBy("FallStatus").count().show()

+----------+-------+
|FallStatus|  count|
+----------+-------+
|   GENESEN|3471830|
| GESTORBEN|  88350|
+----------+-------+



Die meisten Machine Learning Algorithmen arbeiten am Besten wenn die Nummer der Samples in allen Klassen ungefähr die selbe größe haben. Dies konnte auch im Zuge dieser Arbeit bei den unterschiedlichen Regressions-Modellen festgestellt werden. Da die einzelnen Modelle versuchen den Fehler zu reduzieren, haben alle Modelle am Ende für einen Datensatz nur die Klasse Genesen geliefert, da hier die Wahrscheinlichkeit am größten war korrekt zu liegen. 
Um diese Problem zu lösen gibt es zwei Möglichkeiten: Under- und Oversampling. Beides fällt unter den Begriff Resampling
Beim Undersampling werden aus der Klasse mit den meisten Instanzen, Datensätze gelöscht, wohingegen beim Oversampling, der Klasse mit den wenigsten Isntanzen, neue Werte hinzugefügt werden. (Will Badr 2019; Tara Boyle 2019)
Da in diesem Fall ausreichend Datensätze vorhanden sind, bietet sich Ersteres an.

In [10]:
# Ermittlung der Anzahl dr Verstorbenen
dfGestorben = dfsample.filter(dfsample.FallStatus == "GESTORBEN")
anzahlGestorben = dfGestorben.count()
print("Anzahl Gestorben : %s" % anzahlGestorben)

Anzahl Gestorben : 88350


In [11]:
# Ermittlung des Verhätlnisses von Verstorben und Gensen
dfGenesen = dfsample.filter(dfsample.FallStatus == "GENESEN")
anzahlGenesen = dfGenesen.count()
print("Anzahl Genesen : %s" % anzahlGenesen)

ratio = anzahlGestorben / anzahlGenesen
print("Verhältnis : %s" % ratio)



Anzahl Genesen : 3471830
Verhältnis : 0.02544767456931935


In [12]:
# Ziehen eines Samples mit der näherungsweise selben Anzahl wie Verstorbene
dfGenesenSample = dfGenesen.sample(fraction=ratio, seed=12345)

In [13]:
dfGesamtSample = dfGestorben.union(dfGenesenSample)
# Kontrolle
dfGesamtSample.groupBy("FallStatus").count().show()

+----------+-----+
|FallStatus|count|
+----------+-----+
|   GENESEN|88520|
| GESTORBEN|88350|
+----------+-----+



### Splitten in Trainings und Testdaten

In [14]:
splits = dfGesamtSample.randomSplit([0.8, 0.2], 345678)
trainingData = splits[0]
testData = splits[1]

### Aufbau des Feature-Vectors

In [15]:
assembler =  VectorAssembler(outputCol="features", inputCols=["GeschlechtIndex","AltersgruppeIndex", "LandkreisIndex","BundeslandIndex"])

## Modellierung
### Generalized linear regression
Da es sich um ein binäres Klassifikationsproblem handelt, ist in diesem Fall die Familily binominal anzugeben.

In [16]:
glr = GeneralizedLinearRegression(featuresCol="features", labelCol="FallStatusIndex",family="binomial", maxIter=100)

### Pipeline

In [17]:
pipeline = Pipeline(stages=[assembler,glr])

### Evaluator
Für die spätere Cross-Validaton wird ein Evaluator benötigt. Letzterer ist zu wählen, abhängig von dem jeweilligen Modell und Anwendungsfall. (Apache Spark 2020a) In diesem Fall wird ein BinaryClassificationEvaluator angewendet. Dieser eignet sich besonders für binäre Werte. (Apache Spark 2021a) Da Geschlecht, in diesem Fall, der FallStatus 0 oder 1 annehmen kann, bietet er sich hier besonders an.

In [18]:
# Definition des Evaluators
evaluator= BinaryClassificationEvaluator(labelCol="FallStatusIndex",rawPredictionCol="prediction")

### Parametertuning
Eine wichtige Aufgabe beim Machine Learning ist die Auswahl des geeigneten Modells bzw. die passenden Paramter für ein Modell herauszufinden. Letzteres wird auch Parametertuning genannt. Die in Pyspark enthaltene MLLib bietet speziell hierfür ein entsprechende Tooling. Und zwar kann ein CrossValidator bzw. ein TrainValidationSplit verwendet werden. Voraussetzung sind ein Estimator (ein Modell oder eine Pipeline), ein Paramter-Grid und eine Evaluator. Dies ist auch im Zusammenhang mit dem Thema Cross-Validation zu sehen. (Apache Spark 2020a)

In [19]:
paramGrid = ParamGridBuilder()\
    .addGrid(glr.regParam, [0.1, 0.01]) \
    .addGrid(evaluator.metricName, ["areaUnderPR", "areaUnderROC"])\
    .build()

### Cross-Validation

In [20]:
# Definition des Cross-Validators 
# num-Folds gibt an in wie viele Datensatz-Paare die Datensätze aufgeteilt werden.
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2,
                          parallelism=2)

#### Training

In [21]:
# Anpassung des Modells und Auswahl der besten Parameter
cvModel = crossval.fit(trainingData)

In [22]:
# Ermitteln der Paramter
print(cvModel.explainParams())

estimator: estimator to be cross-validated (current: Pipeline_cc6cc3d8d90d)
estimatorParamMaps: estimator param maps (current: [{Param(parent='GeneralizedLinearRegression_2b2b555ebb6a', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='BinaryClassificationEvaluator_5ba4503c5374', name='metricName', doc='metric name in evaluation (areaUnderROC|areaUnderPR)'): 'areaUnderPR'}, {Param(parent='GeneralizedLinearRegression_2b2b555ebb6a', name='regParam', doc='regularization parameter (>= 0).'): 0.1, Param(parent='BinaryClassificationEvaluator_5ba4503c5374', name='metricName', doc='metric name in evaluation (areaUnderROC|areaUnderPR)'): 'areaUnderROC'}, {Param(parent='GeneralizedLinearRegression_2b2b555ebb6a', name='regParam', doc='regularization parameter (>= 0).'): 0.01, Param(parent='BinaryClassificationEvaluator_5ba4503c5374', name='metricName', doc='metric name in evaluation (areaUnderROC|areaUnderPR)'): 'areaUnderPR'}, {Param(parent='GeneralizedLinearRegression

### Testen des Modells

In [23]:
predictions = cvModel.transform(testData)
predictions.show()

+-----------------+---------------+----------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+------------------+
|       Bundesland|BundeslandIndex|       Landkreis|LandkreisIndex|Altersgruppe|AltersgruppeIndex|Geschlecht|GeschlechtIndex|FallStatus| Falldatum|FallStatusIndex|            features|        prediction|
+-----------------+---------------+----------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+------------------+
|Baden-Württemberg|              8|     LK Biberach|          8426|     A60-A79|              2.0|         M|            1.0| GESTORBEN|2020-04-14|            1.0|[1.0,2.0,8426.0,8.0]|0.5595699456378977|
|Baden-Württemberg|              8|     LK Biberach|          8426|     A60-A79|              2.0|         M|            1.0| GESTORBEN|2020-12-08|            1.0|[1.0,2.0,8426.0,8.0]|

In [24]:
# Kontrolle der Predictions
predictions.groupBy("FallStatusIndex", "prediction").count().show()

+---------------+-------------------+-----+
|FallStatusIndex|         prediction|count|
+---------------+-------------------+-----+
|            1.0| 0.7298103946620641|    8|
|            1.0| 0.7779923193739018|   17|
|            1.0|0.45490438597701927|    6|
|            1.0|  0.793201978837983|    1|
|            1.0|0.12168261431147534|    2|
|            1.0| 0.4891389508233265|    4|
|            1.0| 0.4592598156159911|    8|
|            1.0|0.43502771280434693|    1|
|            1.0|0.09440472427494288|    1|
|            1.0| 0.7796897368271406|   17|
|            1.0|0.46541091727867817|    2|
|            1.0| 0.5292418643998077|    6|
|            1.0| 0.7281522149433414|    7|
|            1.0|0.09677339964322573|    1|
|            1.0| 0.4622402326327348|    4|
|            1.0| 0.7348299928404526|    5|
|            1.0| 0.4597064879077278|    3|
|            1.0| 0.6909748816047612|   85|
|            1.0| 0.5822604950071255|    9|
|            0.0| 0.702641696253

## Modell - Evaluation

Area Under PR / Area under ROC

In [25]:
accuracy = evaluator.evaluate(predictions)
print("Test Error",(1.0 - accuracy))

Test Error 0.18102349295627518


### BinaryClassificationMetrics

Bei dem untersuchten Label (FallStatus mit den Ausprägungen Verstorben und Genesen) handelt es sich um ein einen BinaryClasificator. Er kann die Werte 0 und 1 annehmen. Für die Modellevaluation sind daher die BinaryClassificationMetrics zu verwenden. (Apache Spark 2021i)

In [26]:
predictionAndLabels = predictions.select("prediction", "FallStatusIndex").rdd.map(lambda p: [p[0], p[1]]) # Map to RDD prediction|label

In [27]:
# Instanzieire das BinaryClassificationMetrics-Objekt
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Fläche unter der Precision-recall Curve

print("Area under PR = %s" % metrics.areaUnderPR)

# Fläche unter der ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

Area under PR = 0.694023901437784
Area under ROC = 0.8189713841248136


### Multiclass classification Metrics
Da hier Regressions und keine Binären Daten geliefert werden, kann hier  nicht ohne weiteres eine Mulitclass Classification Metric verwendet werden.